In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import pandas as pd
from sklearn.preprocessing import MinMaxScaler


# **Constants** for counting chunks, lines in config_file and for the config_file

In [ ]:
chunk_num_custom = 1
c = 1
config = pd.DataFrame(columns=["result","filename"])

Two functions for dataset formation: the first one to convertion milliseconds to seconds and count from 1 to 9 seconds as integers in each chunck of frames, the second for dividing on chunks

In [ ]:
def convert_timestamp_to_seconds(df, name):
    # Convert 'Timestamp' from milliseconds to seconds
    df['seconds'] = df[' Timestamp '] / 1000

    # Normalize the time to count from 1 to 9 seconds
    min_time = df['seconds'].min()
    df[' Timestamp '] = ((df['seconds'] - min_time) % 9 + 1).astype(int)
    df = df.drop('chunk',axis=1)
    df = df.drop('seconds',axis=1)
    df.to_csv(name, index=False) 
    
    return df[[' Timestamp ']]

In [ ]:
def process_and_save_chunks(c, chunk_num_custom, className):
    # Assuming the timestamp column is named 'timestamp' in milliseconds
    df['seconds'] = df[' Timestamp '] / 1000  # Convert milliseconds to seconds
    # Group into chunks of 10 seconds
    df['chunk'] = (df['seconds'] // 9).astype(int)
    # Iterate through each chunk and save to a separate CSV file
    for chunk_num, chunk_data in df.groupby('chunk'):
        # Define the filename for the chunk
        output_file = f'{chunk_num_custom}_{className}.csv'
        chunk_num_custom = chunk_num_custom + 1;
        config.loc[c,['result','filename']] = className,output_file
        convert_timestamp_to_seconds(chunk_data, output_file)
        c = c+1
        
        print(f'Saved: {output_file}')
    return c, chunk_num_custom

# The main code part, where we are getting the file and make sume manipulations with data:

    1. Drop columns: "Frame number "," Score (right) "," Score (left) ", " Pupil area (right) px ", 
    " Pupil area (left) px "," Scene picture number "," System time Year:Month:Day:Hour:Minute:Second:Milliseconds ",
    " Cornea center X (right) ", " Cornea center Y (right) "," Cornea center Z (right) "," Cornea center X (left) ", 
    " Cornea center Y (left) "," Cornea center Z (left) "," Calibration error ", " Client event"
    
    2. "Eye event" column with empty value is changed on "NA" value, because glasses cannot recognize the eye event, but coordinates can be good for classification
    
    3. We have dropped some rows base on "Score", if the score is less of this value, it is dropped. After our own research we found that for each activity this value is different. For reading: 0.3. For walking: 0.17. For playing: 0.23.
    After it the column with "Score" value was also dropped.

    4. ' Pupil area (right) sq mm ', ' Pupil area (left) sq mm ' were normalized from 0 to 1 by MinMaxScaler

    5. Finally, data was divided on chancks and timestamp data was reclassified. Everything was saved and data with file name and class was written in configuration file

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        input_file = os.path.join(dirname, filename)
        print(filename)
        className = filename.split("_")[0]
        df = pd.read_csv(input_file)
        COL_LIST = ["Frame number "," Score (right) "," Score (left) ", " Pupil area (right) px ", " Pupil area (left) px "," Scene picture number "," System time Year:Month:Day:Hour:Minute:Second:Milliseconds "," Cornea center X (right) ", " Cornea center Y (right) "," Cornea center Z (right) "," Cornea center X (left) ", " Cornea center Y (left) "," Cornea center Z (left) "," Calibration error ", " Client event"]
        df = df.drop(COL_LIST,axis=1)
        value = 0
        # change value for dropping depending on the class
        if(className == 'reading'): 
            value = 0.30
        elif (className == 'walking'):
            value = 0.17
        elif (className == 'playing'):
            value = 0.23
        score = df[(df[' Score '] <= value)].index
        df.drop(score, inplace=True)
        df[' Eye event '] = df[' Eye event '].replace("  ", " NA ")
        df = df.drop(" Score ",axis=1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        columns_to_normalize = [' Pupil area (right) sq mm ', ' Pupil area (left) sq mm ']
        df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
        df[columns_to_normalize] = df[columns_to_normalize].round(2)
        c, chunk_num_custom = process_and_save_chunks(c, chunk_num_custom, className)

In [ ]:
config.to_csv("config_data.csv", index=False) 